In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok -q
!pip install pygltflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ['NGROK_AUTHTOKEN']='2puCttnzpFo3GzXq1FthSd3LGP0_2HPUXbNg7dDp5RAQw2Gpn'

In [5]:
from pygltflib import GLTF2
import os
import shutil

def extract_textures_and_copy_model(glb_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    gltf = GLTF2().load(glb_path)

    glb_filename = os.path.basename(glb_path)
    copied_glb_path = os.path.join(output_dir, glb_filename)
    shutil.copy2(glb_path, copied_glb_path)
    print(f"📦 Copied model to {copied_glb_path}")

    with open(glb_path, 'rb') as f:
        content = f.read()

    def get_bin_chunk():
        magic = int.from_bytes(content[0:4], 'little')
        assert magic == 0x46546C67  # b'glTF'
        json_len = int.from_bytes(content[12:16], 'little')
        json_type = content[16:20]
        assert json_type == b'JSON'

        bin_offset = 20 + json_len
        bin_len = int.from_bytes(content[bin_offset:bin_offset+4], 'little')
        bin_type = content[bin_offset+4:bin_offset+8]
        assert bin_type == b'BIN\x00'

        return content[bin_offset+8 : bin_offset+8+bin_len]

    bin_chunk = get_bin_chunk()
    base_name = os.path.splitext(glb_filename)[0]

    for i, image in enumerate(gltf.images):
        if image.bufferView is None:
            print(f"Skipping image {i} (no bufferView)")
            continue

        buffer_view = gltf.bufferViews[image.bufferView]
        offset = buffer_view.byteOffset or 0
        length = buffer_view.byteLength

        image_data = bin_chunk[offset : offset + length]
        ext = 'png' if image.mimeType == 'image/png' else 'jpg'
        out_path = os.path.join(output_dir, f'{base_name}_texture_{i}.{ext}')

        with open(out_path, 'wb') as f:
            f.write(image_data)
            print(f"Saved texture to {out_path}")

extract_textures_and_copy_model(
    glb_path="/content/decimated_texture_95.glb",
    output_dir="/content/zip"
)

📦 Copied model to /content/zip/decimated_texture_95.glb
✅ Saved texture to /content/zip/decimated_texture_95_texture_0.png


In [6]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                abs_path = os.path.join(root, file)
                rel_path = os.path.relpath(abs_path, folder_path)  # relative path inside zip
                zipf.write(abs_path, arcname=rel_path)
    print(f"Zipped folder to: {zip_path}")

zip_folder(
    folder_path="/content/zip",
    zip_path="/content/decimated_texture_95.zip"
)

📦 Zipped folder to: /content/decimated_texture_95.zip


In [7]:
from fastapi import FastAPI
from fastapi.responses import FileResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import os
from fastapi.responses import PlainTextResponse

nest_asyncio.apply()

app = FastAPI()
PORT = 8000

FILE_PATH = "/content/decimated_texture_95.zip"

@app.get("/")
def read_root():
    return PlainTextResponse("Welcome! FastAPI server is live. Go to /get-3d-file to download the 3D file.")

@app.get("/get-3d-file")
def get_3d_file():
    if not os.path.exists(FILE_PATH):
        return {"error": "3D file not found."}
    return FileResponse(
        path=FILE_PATH,
        media_type="application/octet-stream",
        filename=os.path.basename(FILE_PATH)
    )

if __name__ == "__main__":
    public_url = ngrok.connect(PORT)
    print(f"Public URL: {public_url}/get-3d-file")

    uvicorn.run(app, host="0.0.0.0", port=PORT)


INFO:     Started server process [599]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🌐 Public URL: NgrokTunnel: "https://10c1c631d4d4.ngrok-free.app" -> "http://localhost:8000"/get-3d-file
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET /get-3d-file HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET /get-3d-file HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET /get-3d-file HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:4018:afdf:2da4:d8c9:b22c:c71:0 - "GET /get-3d-file HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [599]
